In [27]:
xlim,ylim = 4,4
import copy
class GameState:
    def __init__(self):
        """
        Attributes
        ----------
        _parity: bool
            keep track of player turn.
        
        _player_loc: list(tuple)
            keep track of player current position
            
        _board: list(list)
            Board representation
        """
        self._parity = 0
        self._player_location = [None,None]
        self._board = [[0]*ylim for _ in range(xlim)]
#         self._board[-1][-1] = 1
    
    
    def get_legal_moves(self):
        """
        Return a list of all legal moves available for current player.
        """
        loc = self._player_location[self._parity]
        if not loc:
            return self._get_blank_spaces()
        moves = []
        path = [(-1,-1),(-1,0),(-1,1),(0,-1),
                (0,1),(1,-1),(1,0),(1,1)]
        for i,j in path:
            x,y = loc
            while 0 <= x+i < xlim and 0 <= y+j < ylim:
                x,y = x+i,y+j
                if self._board[x][y]:
                    break
                moves.append((x,y))
        return moves
        
    
    def _get_blank_spaces(self):
        """
        Return a list of blank spaces.
        """
        return [(x,y) for x in range(xlim) for y in range(ylim) 
                if self._board[x][y] == 0]
        
        
    def forecast_move(self,move):
        """
        Return a new board with specified move applied to the current player
        
        Parameters
        ----------
        move: tuple
            The target position player want to move.
        """
        if move not in self.get_legal_moves():
            raise RuntimeError("illegal move")
        newBoard = copy.deepcopy(self)
        newBoard._board[move[0]][move[1]] = 1
        newBoard._player_location[self._parity] = move
        newBoard._parity ^= 1
        return newBoard
    

def minimax_decision(gameState, depth):
    """ Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player.
    
    You can ignore the special case of calling this function
    from a terminal state.
    """
    
    best_score = float("-inf")
    best_move = None
    loc = gameState._player_location[gameState._parity]
    if not loc:
        moves = [(x,y) for x in range((xlim//2+1)) for y in range((ylim//2+1))]
        for m in moves:
            v = min_value(gameState.forecast_move(m), depth-1, float("-inf"), float("inf"))
            if v > best_score:
                best_score = v
                best_move = m
    else:
        for m in gameState.get_legal_moves():
            v = min_value(gameState.forecast_move(m), depth-1, float("-inf"), float("inf"))
            if v > best_score:
                best_score = v
                best_move = m
    return best_move

def min_value(gameState, depth, alpha, beta):
    """ Return the value for a win (+1) if the game is over,
    otherwise return the minimum value over all legal child
    nodes.
    """
    if terminal_test(gameState):
        return 1
    v = float("inf")
    for m in gameState.get_legal_moves():
        v = min(v, max_value(gameState.forecast_move(m), depth-1, alpha, beta))
        beta = min(beta, v)
        if beta <= alpha:
            break
    return v


def max_value(gameState, depth, alpha, beta):
    """ Return the value for a loss (-1) if the game is over,
    otherwise return the maximum value over all legal child
    nodes.
    """
    if terminal_test(gameState):
        return -1
    v = float("-inf")
    for m in gameState.get_legal_moves():
        v = max(v, min_value(gameState.forecast_move(m), depth-1, alpha, beta))
        alpha = max(alpha, v)
        if beta <= alpha:
            break
    return v

call_counter = 0
def terminal_test(gameState):
    """ Return True if the game is over for the active player
    and False otherwise.
    """
    global call_counter
    call_counter+=1
    return not bool(gameState.get_legal_moves())  # by Assumption 1

In [25]:
import time
def game():
    g = GameState()
    depth_limit = 2
    while bool(g.get_legal_moves()):
        start = time.time()
        m = minimax_decision(g, depth_limit)
        end = time.time()
        print("Player 1 ",m,end-start)
        g = g.forecast_move(m)
        if not bool(g.get_legal_moves()):
            if g._parity:
                print("Player 1 win")
            else:
                print("Player 2 win")
            break
        print(g.get_legal_moves())
        x,y = map(int,input(">").split())
        g = g.forecast_move((x,y))
        

In [29]:
game()

Player 1  (0, 0) 211.27203845977783
[(0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]
>1 1
Player 1  (0, 1) 11.837921142578125
[(0, 2), (1, 0), (1, 2), (1, 3), (2, 0), (2, 1), (3, 1), (2, 2), (3, 3)]
>0 2
Player 1  (1, 2) 0.6832032203674316
[(0, 3), (1, 3)]
>1 3
Player 1  (3, 0) 0.08480668067932129
[(0, 3), (2, 2), (3, 1), (2, 3), (3, 3)]
>3 1
Player 1  (2, 1) 0.012991189956665039
[(2, 0), (2, 2), (3, 2), (3, 3)]
>3 2
Player 1  (2, 2) 0.002019166946411133
[(2, 3), (3, 3)]
>2 3
Player 1  (3, 3) 0.0
Player 1 win
